In [32]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Here's the source for the dataset and data dictionary
path = "https://gist.githubusercontent.com/ryanorsinger/55ccfd2f7820af169baea5aad3a9c60d/raw/da6c5a33307ed7ee207bd119d3361062a1d1c07e/student-mat.csv"

df, X_train_explore, \
    X_train_scaled, y_train, \
    X_validate_scaled, y_validate, \
    X_test_scaled, y_test = wrangle.wrangle_student_math(path)

### SelectKBest

Select the K best features using a statistical test to compare each X with y and find which X's have the strongest relationship with yt. For regression, use the correlation test (f-regression) to score the relationships.

1. Initialize the f_selector object, setting the parameters, or instructions for the method to follow: "use the f_regression test for scoring the features to return to me the top 10 features", for example.
2. Fit the object to our data. That is, run a correlation test for every X variable with our y variable, and then rank the X variables based on how common are with the y/target variable. Then give me the top 10 features.
3. Use get_support() to get the list of features, and save them to a variable that you can use to filter your dataframe in modeling.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 8 features
f_selector = SelectKBest (f_regression, k=8)

# find the top 8 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not.
feature_mask - f_selector.get_support()

# get list of top K features.
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [ ]:
f_feature

### Recursive Feature Elimination

Recursive Feature Elimination will create a model with all the features, evaluate the performance metrics, find the weakest features, remove it, then create a new model with the remaining features, evaluate the performance metrics, find the weakest feature, remove it, and so on, until it gets down to the number of features you have indicated you want when creating the rfe object. You will also indicate wwhich Machine Learning algorithm you want to use.

1. Initialize the machine learning algorithm, in this case LinearRegression
2. Initialize the RFE object, and provide the ML algorithm object from step 1
3. Fit the RFE object to our data. Doing this will provide us a list of features ( the number we asked for) as well as a ranking of all the featuers.
4. Assign the list of selected features to a variable.
5. Optional: Get ranking of all variables (1 being most important)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with.
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)

#get the mask of the columns selected
feature_mask = rfe.support_

#get list of the column names.
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [ ]:
rfe_feature

In [ ]:
# view list of columns and their ranking

# get the ranks
var_ranks = rfe.ranking_
# get the variable names
var_name = X_train_scaled.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
# sort the df by rank
rfe_ranks_df.sort_values('Rank')

## Exercises

#### 1. Load the tips dataset.

In [1]:
from pydataset import data

In [2]:
tips = data('tips')

####   a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.
  
   

In [3]:
tips['tip_percentage'] = tips.tip / tips.total_bill

In [4]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


#### b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [5]:
tips['price_per_person'] = tips.total_bill / tips['size']

In [53]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


#### c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

I think the most important feature to predict tip amount will be total_bill. 

The most important feature to predict tip percentage will be price_per_person.

#### d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What  are they?


In [81]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   size              244 non-null    int64  
 7   tip_percentage    244 non-null    float64
 8   price_per_person  244 non-null    float64
dtypes: float64(4), int64(1), object(4)
memory usage: 19.1+ KB


In [8]:
from sklearn.model_selection import train_test_split

def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, random_state=seed)
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=seed)
    return train, validate, test

# split into train, validate, test
train, validate, test = train_validate_test_split(tips, tips['tip'], seed=123)

In [11]:
from sklearn.feature_selection import SelectKBest, f_regression

X_train = train.drop(columns=['tip','tip_percentage'])
y_train = train['tip']

X_validate = validate.drop(columns=['tip','tip_percentage'])
y_validate = validate['tip']

X_test = test.drop(columns=['tip','tip_percentage'])
y_test = test['tip']

In [13]:
import sklearn.preprocessing

quant_vars = ['total_bill', 'size', 'price_per_person']

scaler = sklearn.preprocessing.RobustScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(train[quant_vars])

X_train_scaled = scaler.transform(X_train[quant_vars])
X_validate_scaled = scaler.transform(X_validate[quant_vars])
X_test_scaled = scaler.transform(X_test[quant_vars])

In [39]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=['total_bill','size','price_per_person'])
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=['total_bill','size','price_per_person'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=['total_bill','size','price_per_person'])

In [46]:
# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 2 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not.
feature_mask = f_selector.get_support()

# get list of top K features.
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [47]:
f_feature

['total_bill', 'size']

In [49]:
y_train = pd.DataFrame(y_train)

In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [55]:
rfe_feature

['total_bill', 'price_per_person']

In [56]:
# view list of columns and their ranking

# get the ranks
var_ranks = rfe.ranking_
# get the variable names
var_names = X_train_scaled.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
# sort the df by rank
rfe_ranks_df.sort_values('Rank')

,Var,Rank
0,total_bill,1
2,price_per_person,1
1,size,2


#### e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage.  What are they?

In [57]:
X_train = train.drop(columns=['tip','tip_percentage'])
y_train = train['tip_percentage']

X_validate = validate.drop(columns=['tip','tip_percentage'])
y_validate = validate['tip_percentage']

X_test = test.drop(columns=['tip','tip_percentage'])
y_test = test['tip_percentage']

In [69]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

X_train_scaled = scaler.transform(X_train[quant_vars])
X_validate_scaled = scaler.transform(X_validate[quant_vars])
X_test_scaled = scaler.transform(X_test[quant_vars])

In [72]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=['total_bill','size','price_per_person'])
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=['total_bill','size','price_per_person'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=['total_bill','size','price_per_person'])

In [77]:
# find the top 2 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not.
feature_mask = f_selector.get_support()

# get list of top K features.
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [78]:
f_feature

['total_bill', 'price_per_person']

In [79]:
# fit the data using RFE
rfe.fit(X_train_scaled,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [80]:
rfe_feature

['size', 'price_per_person']

#### f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

With SelectKBest the goal is to find and keep the attributes with highest correlation to the target variable and of those features. Recursive Feature Elimination, on the other hand, works by using the output of a machine learning algorithm as the evaluation criteria for eliminating features. Yes.

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [156]:
def select_kbest(X, y, k):
    # parameters: f_regression stats test, give me 2 features
    f_selector = SelectKBest(f_regression, k=k)

    # find the top 2 X's correlated with y
    f_selector.fit(X, y)

    # boolean mask of whether the column was selected or not.
    feature_mask = f_selector.get_support()

    # get list of top K features.
    f_feature = X.iloc[:,feature_mask].columns.tolist()
    
    return f_feature
    
    
    

In [158]:
select_kbest(X_train_scaled, y_train, 2)

['total_bill', 'price_per_person']

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.


In [161]:
def rfe(X, y, k):
    from sklearn.linear_model import LinearRegression
    from sklearn.feature_selection import RFE

    # initialize the ML algorithm
    lm = LinearRegression()

    # create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, k)

    # fit the data using RFE
    rfe.fit(X,y)  

    # get the mask of the columns selected
    feature_mask = rfe.support_

    # get list of the column names. 
    rfe_feature = X.iloc[:,feature_mask].columns.tolist()
    
    return rfe_feature

In [162]:
rfe(X_train_scaled, y_train, 2)

['size', 'price_per_person']

In [125]:
array = np.array([[1, 2, 3, 'hi', 'hello'], [4, 5, 6, 'sup','yup']])

In [126]:
array

array([['1', '2', '3', 'hi', 'hello'],
       ['4', '5', '6', 'sup', 'yup']], dtype='<U21')

In [127]:
df = pd.DataFrame(array, columns=['a','b','c','d','e'])

In [132]:
df[['a','b','c']] = df[['a','b','c']].astype(int)

In [133]:
df.dtypes

a     int64
b     int64
c     int64
d    object
e    object
dtype: object

In [149]:
df.a.dtypes == 'object'

False

In [151]:
quant_values = []
for col in df.columns:
    if df[col].dtypes == 'object':
        quant_values.append(col)

In [152]:
quant_values

['d', 'e', 'quantitative_values']

In [154]:
quant_df = df[quant_values]

In [155]:
quant_df

,d,e,quantitative_values
0,hi,hello,NaN
1,sup,yup,NaN
